In [ ]:
#Importing necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Model
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow_hub as hub
import os
from tensorflow import keras
from keras.preprocessing import image
from matplotlib.pyplot import imread
from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix, classification_report , accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import train_test_split

In [ ]:
#Mounting Google Drive
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [ ]:
#Loading the train dataset
df_train = pd.read_csv('/content/gdrive/MyDrive/train_NoNTTqq.csv')
df_train

,id,ultraviolet_filter,green_filter,red_filter,near_infrared_filter,alpha,delta,redshift,stellar
0,1,17.44385,15.71196,16.14848,15.647619,158.167937,29.746275,0.094857,1
1,2,22.02806,24.01481,21.16334,20.214615,145.916931,38.083063,0.361631,1
2,3,23.07242,21.79252,20.51945,18.159421,245.684677,49.908866,-0.000065,2
3,4,23.45985,23.41583,20.36645,20.220636,204.812750,33.137303,0.643375,1
4,5,23.89627,23.18005,21.12911,19.818470,209.254795,55.296589,0.486448,1
...,...,...,...,...,...,...,...,...,...
134906,134907,19.24538,18.80673,16.41091,15.609013,6.218972,19.934894,0.049424,1
134907,134908,23.41124,22.59072,22.50731,22.023594,152.664791,46.368871,0.610840,3
134908,134909,21.76064,20.16531,20.07795,20.315816,358.506516,42.845364,2.133721,3
134909,134910,18.55473,17.70518,16.67601,15.922038,160.649389,18.329627,0.052593,1


In [ ]:
#Dropping the id column from the train dataframe
df_train.drop(columns='id', inplace=True)

In [ ]:
#Replacing each value of stellar column to be in line with SparseCategoricalCrossentropy
df_train['stellar'].replace({
    1:0,
    2:1,
    3:2
}, inplace=True)

In [ ]:
#Loading the test dataset
df_test = pd.read_csv('/content/gdrive/MyDrive/test_SxgqOdc.csv')
df_test

,id,ultraviolet_filter,green_filter,red_filter,near_infrared_filter,alpha,delta,redshift
0,134912,19.63144,17.88840,16.45195,16.620047,336.501421,2.415351,0.000290
1,134913,25.74819,22.10760,19.97196,19.179141,210.286161,-1.336858,0.513781
2,134914,22.38767,20.85446,20.75418,20.073627,262.914770,46.025803,0.985297
3,134915,22.03212,23.15455,21.86528,20.746343,146.381732,38.368224,0.712391
4,134916,25.01815,24.53933,22.03248,22.377272,246.456081,30.515558,0.495552
...,...,...,...,...,...,...,...,...
89936,224848,22.41481,21.55370,19.60544,18.617794,1.108111,30.528644,0.000479
89937,224849,25.69069,22.74517,21.85320,21.419104,155.468306,37.207024,0.649914
89938,224850,20.79857,19.29775,16.87349,16.802314,186.069454,23.731177,0.370957
89939,224851,22.15261,20.10221,19.14552,18.521452,193.814800,29.493972,0.000959


In [ ]:
#Storing the columns of train dataframe other than stellar in X and storing the stellar column of train dataframe in Y
X = df_train.drop('stellar', axis='columns')
Y = df_train.stellar

In [ ]:
#Train test split
from sklearn.model_selection import train_test_split
x_train, x_val, y_train, y_val = train_test_split(X, Y, test_size = 0.00001)
#To ensure maximum data is used in training, validation is minimised after optimising hyperparameters

In [ ]:
#Dropping the id colum of test dataframe
df_test_p = df_test.drop('id', axis='columns')
df_test_p

,ultraviolet_filter,green_filter,red_filter,near_infrared_filter,alpha,delta,redshift
0,19.63144,17.88840,16.45195,16.620047,336.501421,2.415351,0.000290
1,25.74819,22.10760,19.97196,19.179141,210.286161,-1.336858,0.513781
2,22.38767,20.85446,20.75418,20.073627,262.914770,46.025803,0.985297
3,22.03212,23.15455,21.86528,20.746343,146.381732,38.368224,0.712391
4,25.01815,24.53933,22.03248,22.377272,246.456081,30.515558,0.495552
...,...,...,...,...,...,...,...
89936,22.41481,21.55370,19.60544,18.617794,1.108111,30.528644,0.000479
89937,25.69069,22.74517,21.85320,21.419104,155.468306,37.207024,0.649914
89938,20.79857,19.29775,16.87349,16.802314,186.069454,23.731177,0.370957
89939,22.15261,20.10221,19.14552,18.521452,193.814800,29.493972,0.000959


In [ ]:
#Standard Scaling
from sklearn.preprocessing import StandardScaler
st_x = StandardScaler()
x_train_scaled = st_x.fit_transform(x_train)
x_test_scaled = st_x.transform(df_test_p)
x_val_scaled = st_x.transform(x_val)

In [ ]:
# Decision Tree Classifiers -> We will explain

In [ ]:
#RandomForest Classifier
from sklearn.ensemble import RandomForestClassifier
model1 = RandomForestClassifier(n_estimators=150)
model1.fit(x_train_scaled, y_train)
model1.score(x_val_scaled, y_val)

0.945004447079751

In [ ]:
#Storing the predicted probabilities obtained using RandomForest Classifier for test data
pred_prob_test_rf = np.array(model1.predict_proba(x_test_scaled))
print(pred_prob_test_rf)

[[0.19333333 0.80666667 0.        ]
 [0.98666667 0.01333333 0.        ]
 [0.28666667 0.01333333 0.7       ]
 ...
 [0.96666667 0.03333333 0.        ]
 [0.         1.         0.        ]
 [0.97333333 0.01333333 0.01333333]]


In [ ]:
#LGBM Classifier
import lightgbm as ltb
model2 = ltb.LGBMClassifier(learning_rate=0.09,max_depth=-5,random_state=42)
model2.fit(x_train_scaled,y_train)
model2.score(x_val_scaled, y_val)

0.945375037058998

In [ ]:
#Storing the predicted probabilities obtained using LGBM Classifier for test data
pred_prob_test_lgbm = np.array(model2.predict_proba(x_test_scaled))
print(pred_prob_test_lgbm)

[[0.12319988 0.87381902 0.0029811 ]
 [0.99030242 0.00753548 0.0021621 ]
 [0.26947999 0.01680256 0.71371746]
 ...
 [0.97065841 0.01801679 0.0113248 ]
 [0.00633309 0.99212172 0.00154518]
 [0.99146459 0.00409811 0.0044373 ]]


In [ ]:
#Installing the catboost library
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 76.8 MB 1.3 MB/s 


In [ ]:
#CatBoost Classifier
import catboost as cb
model3 = cb.CatBoostClassifier()
model3.fit(x_train_scaled,y_train)
model3.score(x_val_scaled, y_val)

Learning rate set to 0.10107
0:	learn: 0.9402960	total: 108ms	remaining: 1m 47s
1:	learn: 0.8222216	total: 160ms	remaining: 1m 19s
2:	learn: 0.7292044	total: 221ms	remaining: 1m 13s
3:	learn: 0.6554640	total: 360ms	remaining: 1m 29s
4:	learn: 0.5933275	total: 484ms	remaining: 1m 36s
5:	learn: 0.5417942	total: 622ms	remaining: 1m 43s
6:	learn: 0.4979741	total: 741ms	remaining: 1m 45s
7:	learn: 0.4615191	total: 794ms	remaining: 1m 38s
8:	learn: 0.4303680	total: 849ms	remaining: 1m 33s
9:	learn: 0.4029473	total: 903ms	remaining: 1m 29s
10:	learn: 0.3794749	total: 960ms	remaining: 1m 26s
11:	learn: 0.3595575	total: 1.02s	remaining: 1m 24s
12:	learn: 0.3414552	total: 1.07s	remaining: 1m 21s
13:	learn: 0.3260903	total: 1.13s	remaining: 1m 19s
14:	learn: 0.3124011	total: 1.19s	remaining: 1m 18s
15:	learn: 0.3007084	total: 1.25s	remaining: 1m 17s
16:	learn: 0.2900462	total: 1.31s	remaining: 1m 15s
17:	learn: 0.2811664	total: 1.37s	remaining: 1m 14s
18:	learn: 0.2729873	total: 1.43s	remaining: 

0.9447079750963534

In [ ]:
#Storing the predicted probabilities obtained using CatBoost Classifier for test data
pred_prob_test_cat = np.array(model3.predict_proba(x_test_scaled))
print(pred_prob_test_cat)

[[1.12692823e-01 8.85239051e-01 2.06812572e-03]
 [9.93739720e-01 5.46504020e-03 7.95240130e-04]
 [2.05738071e-01 1.11114329e-02 7.83150496e-01]
 ...
 [9.66987212e-01 2.04503061e-02 1.25624817e-02]
 [4.27445156e-03 9.95183795e-01 5.41752979e-04]
 [9.96438912e-01 1.21370761e-03 2.34738010e-03]]


In [ ]:
#GradientBoosting Classifier
from sklearn.ensemble import GradientBoostingClassifier
gb_clf = GradientBoostingClassifier(n_estimators=100, max_features=2, random_state=0)
gb_clf.fit(x_train_scaled,y_train)
gb_clf.score(x_val_scaled, y_val)

0.9445597391046546

In [ ]:
#Storing the predicted probabilities obtained using GradientBoosting Classifier for test data
pred_prob_test_gb = np.array(gb_clf.predict_proba(x_test_scaled))
print(pred_prob_test_gb)

[[0.08405353 0.90290217 0.0130443 ]
 [0.98227451 0.01252745 0.00519804]
 [0.27665157 0.02962194 0.69372649]
 ...
 [0.96253862 0.02780198 0.0096594 ]
 [0.00923104 0.98836794 0.00240102]
 [0.98683235 0.00663547 0.00653218]]


In [ ]:
#ExtraTrees Classifier
from sklearn.ensemble import ExtraTreesClassifier
etf = ExtraTreesClassifier(n_estimators = 150, criterion = 'entropy')
etf.fit(x_train_scaled,y_train)
etf.score(x_val_scaled, y_val)

0.9447820930922027

In [ ]:
#Storing the predicted probabilities obtained using ExtraTrees Classifier for test data
pred_prob_test_etf = np.array(etf.predict_proba(x_test_scaled))
print(pred_prob_test_etf)

[[0.24       0.76       0.        ]
 [0.99333333 0.00666667 0.        ]
 [0.42       0.02       0.56      ]
 ...
 [0.95333333 0.02666667 0.02      ]
 [0.00666667 0.99333333 0.        ]
 [0.98       0.00666667 0.01333333]]


In [ ]:
#Implementing Snapshot ensemble
from keras.callbacks import Callback
from keras import backend
from keras.models import load_model
from numpy import pi
import math

class SnapshotEnsemble(Callback):

    __snapshot_name_fmt = "snapshot_%d.hdf5"

    def __init__(self, n_models, n_epochs_per_model, lr_max, verbose=1):

        # n_models: number of snapshots
        # n_epochs_per_model: epochs per snapshot
        # lr_max: maximum learning rate

        self.n_epochs_per_model = n_epochs_per_model
        self.n_models = n_models
        self.n_epochs_total = self.n_models * self.n_epochs_per_model
        self.lr_max = lr_max
        self.verbose = verbose
        self.lrs = []

    def cosine_annealing(self, epoch):
        cos_inner = (math.pi * (epoch % self.n_epochs_per_model)) / self.n_epochs_per_model
        return self.lr_max / 2 * (math.cos(cos_inner) + 1)

    def on_epoch_begin(self, epoch, logs={}):
        lr = self.cosine_annealing(epoch)
        backend.set_value(self.model.optimizer.lr, lr)
        self.lrs.append(lr)

    def on_epoch_end(self, epoch, logs={}):
        if (epoch + 1) % self.n_epochs_per_model == 0:
            filename = self.__snapshot_name_fmt % ((epoch + 1) // self.n_epochs_per_model)
            self.model.save(filename)
            if self.verbose:
                print('Epoch %d: snapshot saved to %s' % (epoch, filename))

    def load_ensemble(self):
        models = []
        for i in range(self.n_models):
            models.append(load_model(self.__snapshot_name_fmt % (i + 1)))
        return models

In [ ]:
Snapshot_Ensemble_callback = SnapshotEnsemble(n_models=3, n_epochs_per_model=20, lr_max=0.01)

In [ ]:
#ANN
import tensorflow as tf
from tensorflow import keras

model_ann = keras.Sequential([
    keras.layers.Dense(7, input_shape=(7,), activation='relu'),
    keras.layers.Dense(15, activation='relu'),
    keras.layers.Dense(80, activation='relu'),
    keras.layers.Dense(200, activation='relu'),
    keras.layers.Dense(100, activation='relu'),
    keras.layers.BatchNormalization(),
    keras.layers.Dropout(0.3),
    keras.layers.Dense(40, activation='relu'),
    keras.layers.Dense(20, activation='relu'),
    keras.layers.Dense(3, activation='softmax')
])

model_ann.compile(optimizer='adam',
              loss='SparseCategoricalCrossentropy',
              metrics=['accuracy'])

model_ann.fit(x_train_scaled, y_train, validation_data=(x_val_scaled, y_val), callbacks = [Snapshot_Ensemble_callback], epochs = Snapshot_Ensemble_callback.n_epochs_total)

Epoch 1/60
3795/3795 [==============================] - 18s 4ms/step - loss: 0.3010 - accuracy: 0.9094 - val_loss: 0.2687 - val_accuracy: 0.9330
Epoch 2/60
3795/3795 [==============================] - 19s 5ms/step - loss: 0.2585 - accuracy: 0.9268 - val_loss: 0.2734 - val_accuracy: 0.9240
Epoch 3/60
3795/3795 [==============================] - 29s 8ms/step - loss: 0.2557 - accuracy: 0.9284 - val_loss: 0.2637 - val_accuracy: 0.9224
Epoch 4/60
3795/3795 [==============================] - 23s 6ms/step - loss: 0.2486 - accuracy: 0.9301 - val_loss: 0.2739 - val_accuracy: 0.9274
Epoch 5/60
3795/3795 [==============================] - 26s 7ms/step - loss: 0.2494 - accuracy: 0.9297 - val_loss: 0.2559 - val_accuracy: 0.9206
Epoch 6/60
3795/3795 [==============================] - 20s 5ms/step - loss: 0.2457 - accuracy: 0.9309 - val_loss: 0.2471 - val_accuracy: 0.9298
Epoch 7/60
3795/3795 [==============================] - 23s 6ms/step - loss: 0.2440 - accuracy: 0.9324 - val_loss: 0.2454 - val_ac

In [ ]:
#Obtaining individual model accuracy
models = Snapshot_Ensemble_callback.load_ensemble()
for model in models:
    model.evaluate(x_val_scaled, y_val)

422/422 [==============================] - 1s 2ms/step - loss: 0.2246 - accuracy: 0.9383


In [ ]:
#Snapshot Ensemble of the models
weights = [0.92, 0.93, 0.94]
predictions = []
val_final = np.zeros([len(x_val_scaled), 3])
final = np.zeros([len(x_test_scaled), 3])
for i, model in enumerate(models):
    val_pred = np.array(model.predict(x_val_scaled))
    pred = np.array(model.predict(x_test_scaled))
    val_final = val_final + weights[i] * val_pred
    final = final + weights[i] * pred
for i in range(len(x_val_scaled)):
    predictions.append(np.argmax(val_final[i]))
c = 0
y_val = np.array(y_val)
for i in range(len(x_val_scaled)):
    if predictions[i] == y_val[i]:
        c+=1;
print("Ensemble accuracy: ", c/len(x_val_scaled))

2811/2811 [==============================] - 7s 3ms/step
Ensemble accuracy:  0.9385561814408538


In [ ]:
#Storing the predicted probabilities obtained using ANN for test data
pred_prob_test_ann = model_ann.predict(x_test_scaled)
print(pred_prob_test_ann)

2811/2811 [==============================] - 5s 2ms/step
[[6.2942110e-02 9.3620700e-01 8.5094135e-04]
 [9.8847586e-01 1.0635775e-02 8.8825234e-04]
 [3.1274137e-01 7.7202404e-03 6.7953837e-01]
 ...
 [9.5337498e-01 4.2527270e-02 4.0977774e-03]
 [2.1943133e-02 9.7703892e-01 1.0179415e-03]
 [9.8134065e-01 1.0600814e-02 8.0584576e-03]]


In [ ]:
#Ensembling with experimental weights
pred_ensem_test_final = (3*pred_prob_test_lgbm + 2*pred_prob_test_rf + 5*pred_prob_test_cat + pred_prob_test_etf + 4*pred_prob_test_gb + pred_prob_test_ann)
print(pred_ensem_test_final)

[[0.11902442 0.87688429 0.00409129]
 [0.98706754 0.0109525  0.00197996]
 [0.23926786 0.01972724 0.74100491]
 ...
 [0.96597891 0.02398768 0.0100334 ]
 [0.00578804 0.99291754 0.00129442]
 [0.98384252 0.00434653 0.01181095]]


In [ ]:
pred_ensem_test = []
for i in pred_ensem_test_final:
  pred_ensem_test.append(np.argmax(i)+1)
pred_ensem_test = np.array(pred_ensem_test)
print(pred_ensem_test)

[2 1 3 ... 1 2 1]


In [ ]:
#Converting to dataframe
dfx = pd.DataFrame(list(zip(pred_ensem_test)),
               columns = ['stellar'])

In [ ]:
dfx.insert(loc = 0,
          column = 'id',
          value = df_test['id'])

In [ ]:
dfx.to_csv('SOS_ensemble_final.csv')